In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip yor_path_to_your_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/llama-2-7b-chat-platypus-1k-Baymax10"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

403

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1584

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  LLaMA-2 was trained on approximately 2 trillion tokens.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  LLaMA-2 was trained on approximately 2 trillion tokens.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the__zip_file
!unzip -q Questionscsv.zip -d questions

###Mamba Questions

In [ ]:
# Mamba Questions

questions_file_path = '/content/questions/MambaQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba achieves computational efficiency
without specialized hardware optimizations through several techniques:

1. Simplified architecture: Mamba has a simplified architecture compared to other state-of-the-art transformers like H3 and Transformer++, which
reduces the number of parameters and computations required.
2. Linear projection: Mamba uses linear projections instead of multi-layer perceptrons (MLPs) to reduce the dimensionality of the input data before
feeding it into the SSM. This helps reduce the computational cost associated with MLPs.
3. Nonlinearity: Mamba uses a simple nonlinearity function (SiLU or Swish) instead of more complex activation functions like ReLU or tanh. This
simplifies the computation and allows for faster convergence.
4. Interleaving: Mamba interleaves the H3 block with the MLP block, allowing it to leverage the strengths of both architectures while avoiding their
respective weaknesses.
5. Batch normalization: Mamba uses batch normalization, which helps

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba introduces several innovations to manage long
sequence data processing:

1. Selective State Space Models (SSMs): Mamba replaces traditional Recurrent Neural Networks (RNNs) with SSMs, which allow for context-dependent
reasoning while scaling linearly in sequence length. This enables Mamba to handle long sequences efficiently without sacrificing performance.
2. Linear Time Processing: Unlike other large language models, Mamba processes sequences in linear time, making it faster and more efficient compared
to other approaches.
3. Autoregressive Training: During training, Mamba trains autoregressively, meaning that each element in the sequence is computed sequentially rather
than caching previous elements. This approach allows Mamba to scale better with increasing sequence lengths.
4. Long Context Support: Mamba supports long context sequences up to one million tokens, enabling it to capture complex relationships between distant
parts of the input sequence.


Sources:
/content/pap

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Based on the given text, there
are several ways that Mamba's performance evaluation suggests areas for future research and development:

1. Scalability: Mamba's ability to scale linearly in sequence length during training and inference suggests that there may be opportunities to further
improve its efficiency and reduce computational requirements. This could involve optimizing hardware configurations or developing new algorithms
specifically tailored to large-scale sequential data processing.
2. Contextual reasoning: The success of Mamba in solving complex tasks that rely heavily on contextual reasoning, such as language modeling and
copywriting, highlights the importance of developing models capable of effectively handling long-range dependencies. Future work could focus on
improving the capacity of Mamba or exploring alternative approaches to tackle these challenges.
3. Domain adaptation: As demonstrated by the results presented in the paper, Mamba excels across multiple domains wi

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Dynamic parameter adjustment plays a
crucial role in Mamba's selective state space models. The authors propose a learnable mechanism for selecting parameters in the state space model,
which enables the model to adaptively adjust its parameters based on the input sequence. This allows the model to efficiently capture contextual
relationships in the data and improve its overall performance. Specifically, the authors use a hyperbolic tangent (tanh) function to dynamically
adjust the value of the parameter A, which controls the balance between focusing on the current input and persisting the previous state. By optimizing
this hyperbolic tanh function during training, the model learns to automatically adjust its parameters to effectively capture contextual relationships
in the data.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Li

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba addresses the
inefficiencies of Transformers in moderate to long sequence processing through several improvements over traditional Transformer architectures. One
key innovation is the introduction of selective state spaces, which allow the model to focus on specific parts of the input sequence when computing
attention scores. This allows Mamba to perform content-based reasoning and improve its ability to process longer sequences compared to traditional
Transformers. Additionally, Mamba uses a novel combination of local convolution and multi-head attention, called the "H3" block, which improves the
efficiency of the self-attention mechanism. Overall, these advancements enable Mamba to achieve superior performance on tasks involving moderate to
long sequences compared to previous state-of-the-art models like Transformers and Hyenas.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modelin

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How does Mamba achieve computational efficiency without specialized hardware optimizations?
 response:   Mamba achieves computational efficiency
without specialized hardware optimizations through several techniques:

1. Simplified architecture: Mamba has a simplified architecture compared to other state-of-the-art transformers like H3 and Transformer++, which
reduces the number of parameters and computations required.
2. Linear projection: Mamba uses linear projections instead of multi-layer perceptrons (MLPs) to reduce the dimensionality of the input data before
feeding it into the SSM. This helps reduce the computational cost associated with MLPs.
3. Nonlinearity: Mamba uses a simple nonlinearity function (SiLU or Swish) instead of more complex activation functions like ReLU or tanh. This
simplifies the computation and allows for faster convergence.
4. Interleaving: Mamba interleaves the H3 block with the MLP block, allowing it to leverage the strengths of both architectures whil

In [ ]:
json_file_name = 'MambaQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'MambaQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Parametric Magnon Questions

In [ ]:
# Parametric Magnon Questions

questions_file_path = '/content/questions/ParametricMagnonQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Materials commonly used to build
quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Magnon nonlinearities enhance quantum transduction by allowing for more
precise control over the interaction between microwaves and spin qubits. Unlike linear magnon dynamics, which rely solely on the collective spin
excitations in magnetic materials, nonlinear magnonics enables the manipulation of individual magnons through their intrinsic nonlinear properties.
This opens up new possibilities for tailoring the spin-magnon coupling strength and selectively controlling the transduction process. By leveraging
these capabilities, the proposed hybrid quantum transducer can significantly improve upon existing approaches relying solely on linear magnon
dynamics.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Parametric magnonics introduces an innovative
approach in quantum transducers by utilizing nonlinear magnon dynamics to enhance microwave transduction to spin qubits. Unlike traditional linear
magnon transducers, which rely solely on the dipole interaction between the spin centers and the linear magnons, the proposed hybrid system employs
parametric processes at the core to selectively turn the spin-magnon coupling "on" and "off." This novel mechanism offers improved protection against
resonant magnon noise-induced errors, making it a promising solution for scaling up quantum computing architectures.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Introducing nonlinear magnonics into quantum computing
systems can significantly impact their performance and capabilities. Nonlinear magnonics refers to the study of how magnetic materials interact with
light through magneto-optic effects, such as magneto-optic Kerr effect (MOKE) and magneto-optic Faraday rotation (MOFR). These effects allow for
manipulation of the material's optical properties by applying external magnetic fields.

In quantum computing applications, nonlinear magnonics can be used to improve the efficiency and accuracy of quantum operations by leveraging the
unique properties of magnonic materials. For example, by incorporating nonlinear magnonics into quantum error correction codes, it may be possible to
reduce errors caused by noise in the quantum system. Additionally, nonlinear magnonics can enable new types of quantum gates and operations that
cannot be achieved using conventional methods.

One potential application of nonlinear magnonics in quantum computing i

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications that could potentially
benefit from magnon nonlinearities in quantum systems include:

1. Quantum communication networks: Nonlinear magnonics could enable more efficient transfer of quantum information over longer distances through
optimized magnon-based transducers.
2. Quantum computing architectures: Hybrid systems combining linear and nonlinear magnonics may lead to improved performance in terms of coupling
strengths and cooperativity, ultimately paving the way towards larger-scale quantum computations.
3. Spintronic devices: Harnessing the power of magnon nonlinearities could revolutionize the development of spintronic devices such as
magnetoresistive random access memory (MRAM) cells, which rely heavily on spin currents.
4. Energy storage and conversion: Magnon-based energy harvesters and generators could become viable options for capturing and converting mechanical
energy, leading to more sustainable and efficient energy management systems.
5. Medical imagin

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What materials are typically used to construct quantum transducers in hybrid quantum systems?
 response:   Materials commonly used to build
quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond.


Q1: How do magnon nonlinearities enhance quantum transduction?
 response:   Magnon nonlinearities enhance quantum transduction by allowing for more
precise control over the interaction between microwaves and spin qubits. Unlike linear magnon dynamics, which rely solely on the collective spin
excitations in magnetic materials, nonlinear magnonics enables the manipulation of individual magnons through their intrinsic nonlinear properties.
This opens up new possibilities for tailoring the spin-magnon coupling strength and selectively controlling the transduction process. By leveraging
these capabilities, the proposed hybrid quantum transducer can significantly improve upon existing approaches relying solely on linear magn

In [ ]:
json_file_name = 'ParametricMagnonQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'ParametricMagnonQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Quantum Mechanics Questions

In [ ]:
# Quantum Mechanics Questions

questions_file_path = '/content/questions/QuantumMechanicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics plays a crucial role in modern technology due
to its unique properties, such as superposition, entanglement, and wave-particle duality. These properties enable quantum systems to perform tasks
faster than their classical counterparts, leading to significant improvements in various fields. Here are some examples of how quantum mechanics
influences modern technology:

1. Supercomputing: Quantum computers use quantum bits (qubits) instead of traditional bits to process information. Qubits can exist in multiple states
simultaneously, allowing quantum computers to solve complex problems exponentially faster than classical computers. For instance, Shor's algorithm for
factorizing large numbers can break RSA encryption used in secure online transactions within hours on a single quantum computer, whereas it would take
millions of years for a classical computer.
2. Cryptography: Quantum key distribution (QKD) utilizes entangled particles to create an unhackable code for secu

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics has greatly influenced several new fields of study,
including:

1. Quantum Information Theory: This field focuses on the application of quantum mechanical concepts to information processing and storage. It has led
to the development of quantum algorithms, quantum cryptography, and quantum teleportation.
2. Quantum Thermodynamics: This field studies the behavior of quantum systems under different temperature conditions. It has helped researchers
understand the fundamental limits of energy efficiency in various quantum systems.
3. Condensed Matter Physics: Quantum mechanics plays a crucial role in understanding the behavior of materials at low temperatures. By studying their
properties, researchers have discovered exotic phases of matter, such as superconductors, superfluids, and graphene.
4. Statistical Mechanics: This field deals with the statistical interpretation of thermodynamic properties of systems. Quantum mechanics has enabled
researchers to extend statistica

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  There are several fundamental questions about quantum
mechanics that remain open, some of which include:

1. Interpretation of Quantum Mechanics: Despite significant efforts over the past century, there is still no consensus on the proper interpretation of
quantum mechanics. Different approaches such as Copenhagen interpretation, Many Worlds Interpretation, and Relational Quantum Mechanics continue to be
discussed and debated.
2. Quantum Measurement Problem: The act of measurement in quantum mechanics is still poorly understood. It is unclear what exactly happens when a
measurement is made, whether it collapses the wave function, or if it simply reveals preexisting properties.
3. Non-Locality and Bell Inequality Violation: Quantum mechanics predicts non-local behavior, where measurements performed on one system can
instantaneously affect another distant system. This violates Bell's inequality, which suggests locality must hold true. However, experiments have
repeatedly confirmed the 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Exploring quantum mechanics contributes significantly to
theoretical physics by providing insights into the behavior of matter and energy at the atomic and subatomic level. Quantum mechanics offers a
framework for understanding the properties and interactions of particles such as electrons, protons, and photons, which are essential components of
all matter. By studying these particles and their interactions, physicists gain a deeper understanding of the fundamental forces of nature, including
gravity, electromagnetism, and the strong and weak nuclear forces. This knowledge enables researchers to develop new technologies based on quantum
mechanics, such as transistors, lasers, magnetic resonance imaging (MRI), and computer chips. Additionally, exploring quantum mechanics helps refine
our understanding of space and time, leading to breakthroughs in areas like cosmology and astrophysics. Overall, the study of quantum mechanics
continues to play a crucial role in advancing our comprehens

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental advances in verifying quantum theory include the
ability to manipulate and measure quantum states with greater precision, demonstration of quantum teleportation and entanglement between particles
separated by large distances, observation of quantum phase transitions, and the implementation of quantum error correction codes. Additionally,
researchers have developed techniques to cool atoms to near absolute zero temperatures, allowing for precise measurements of quantum systems and
observations of quantum phenomena under controlled conditions.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
  Future developments in quantum technology include the potential for
superfast quantum computers, unbreakable quantum cryptography, and ultrasensitive quantum sensors. Additionally, researchers continue to explore the
interplay between quantum mechanics and black-hole physics, as well

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the significance of quantum mechanics in modern technology?
 response:   Quantum mechanics plays a crucial role in modern technology due
to its unique properties, such as superposition, entanglement, and wave-particle duality. These properties enable quantum systems to perform tasks
faster than their classical counterparts, leading to significant improvements in various fields. Here are some examples of how quantum mechanics
influences modern technology:

1. Supercomputing: Quantum computers use quantum bits (qubits) instead of traditional bits to process information. Qubits can exist in multiple states
simultaneously, allowing quantum computers to solve complex problems exponentially faster than classical computers. For instance, Shor's algorithm for
factorizing large numbers can break RSA encryption used in secure online transactions within hours on a single quantum computer, whereas it would take
millions of years for a classical computer.
2. Cryptography: Quantum key di

In [ ]:
json_file_name = 'QuantumMechanicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'Quantum MechanicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Qubit Teleportation Questions

In [ ]:
# Qubit Teleportation Questions

questions_file_path = '/content/questions/QubitTeleportationQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the article, the main components used in
quantum networks for teleportation are:

* NV centres in diamonds as communication qubits
* Nearby 13C nuclear spins as memory qubits
* Optical connections between nodes
* Solid-state spin qubits

These components enable the establishment of remote entanglement between neighboring nodes and allow for the reliable transfer of quantum information
across lossy network connections.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Entanglement establishment between distant nodes in a
quantum network involves several steps. Firstly, entanglement swapping protocols are used to create entanglement between neighboring nodes. This
process involves applying a specific sequence of quantum gates to the qubits involved, which creates a shared entangled state between them. Next,
entanglement swapping is performed on the middle node, where the entangled state is transferred from one node to another through a series of Bell
measurements. Finally, the entangled state is stored in a memory qubit located on the middle node.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory qubits play a crucial role in the
process of quantum teleportation in a network by storing the entangled state generated between neighboring nodes during the entanglement swapping
protocol. This stored entangled state serves as the resource needed for teleportation, allowing the teleported qubit to be reconstructed at the
destination node with fidelity above the classical bound. Additionally, memory qubits provide another level of redundancy against errors caused by
noise in the system, which helps improve the overall fidelity of the teleported qubit.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications of quantum teleportation in
quantum networks include:

* Multi-node protocols and applications: Teleportation-based protocols could enable secure communication among multiple nodes in a quantum network,
enhancing its overall security and reliability.
* Distributed quantum computing: By connecting multiple nodes via teleportation, it might become possible to distribute quantum computations across
the network, leading to faster computation speeds and increased computational capacity.
* Secure quantum communication: Quantum teleportation could play a crucial role in securing quantum communication over long distances by allowing for
the reliable transfer of quantum information between non-adjacent nodes.
* Quantum metrology: Quantum teleportation could enhance precision measurements in quantum metrology by enabling the sharing of quantum information
between distant nodes, leading to improved accuracy and sensitivity.


Sources:
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Real-time feed-forward significantly improves the
quality of quantum teleportation by allowing for more accurate state recovery on the receiver side. By applying the feed-forward gate operation
immediately after the Bell-state measurement (BSM), the error caused by the finite coherence time of the qubits involved in the teleportation process
can be reduced. This results in higher fidelities for the teleported state, making it closer to the ideal case where the state is perfectly recovered
on the receiver side.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
  Technical
advancements required to achieve efficient quantum teleportation between non-neighboring nodes include:

* Improved remote entanglement generation techniques to increase the quality of shared entanglement between pairs of nodes.
* Enhanced memory qubit readou

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What are the main components used in quantum networks for teleportation?
 response:   According to the article, the main components used in
quantum networks for teleportation are:

* NV centres in diamonds as communication qubits
* Nearby 13C nuclear spins as memory qubits
* Optical connections between nodes
* Solid-state spin qubits

These components enable the establishment of remote entanglement between neighboring nodes and allow for the reliable transfer of quantum information
across lossy network connections.


Q1: How is entanglement established between distant nodes in a quantum network?
 response:   Entanglement establishment between distant nodes in a
quantum network involves several steps. Firstly, entanglement swapping protocols are used to create entanglement between neighboring nodes. This
process involves applying a specific sequence of quantum gates to the qubits involved, which creates a shared entangled state between them. Next,
entanglement swapping is performed 

In [ ]:
json_file_name = 'QubitTeleportationQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QubitTeleportationQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Variance Based Questions

In [ ]:
# Variance Based Questions

questions_file_path = '/content/questions/VarianceQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The focus of variance-based sensitivity analysis in
quantum memory is to determine the sensitivity of a quantum memory implementation to changes in intrinsic and extrinsic parameters while keeping
internal system parameters fixed.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In the context of quantum physics, "Lambda-type" refers to a specific type of quantum
memory called /Lambda1-type quantum memory. This type of memory stores and retrieves photonic quantum states by manipulating the population transfer
between two hyperfine levels within an atom ensemble through a carefully controlled laser field. The name "Lambda-type" comes from the mathematical
representation used to describe the process, which involves the Lambda function, denoted by Λ(z).


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory sensitivity analysis is significant because it helps identify areas
where small changes in input parameters could significantly impact the overall performance of a system. By understanding how different components
interact within a complex system, decision-makers can better prioritize their efforts toward optimizing those parts most critical to achieving desired
outcomes. Additionally, by analyzing potential sources of variability early on during development stages allows engineers to design more robust
designs from the start – reducing risk associated with unforeseen issues later down the line. Finally, having knowledge about what factors might cause
problems also enables informed decisions regarding resource allocation when allocating resources towards mitigating risks associated with certain
variables being outside predefined limits.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-b

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental techniques used to analyze memory sensitivity
include:

1. Variance-based sensitivity analysis: This method involves measuring the variance in memory efficiency due to changes in input parameters while
keeping internal system parameters fixed.
2. Two- and three-parameter sensitivity analysis: These methods involve calculating the sensitivity of memory efficiency to correlations between input
parameters.
3. Single-photon detuning sensitivity analysis: This method involves measuring the sensitivity of memory efficiency to changes in single-photon
detuning.
4. Chirp sensitivity analysis: This method involves measuring the sensitivity of memory efficiency to changes in chirp in the optical field.
5. Upper bound on memory fidelity sensitivity analysis: This method involves calculating an upper bound on memory fidelity (or a lower bound on
sensitivity) in the presence of experimental fluctuations and drift.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Gaussian control field relates to memory performance through
its impact on the optical control ﬁeld used in the memory interaction. The Gaussian control ﬁeld parameters, such as pulse area, delay, and duration,
are adjusted to optimize memory efficiency. When these parameters deviate from their optimal values due to drift or improper settings, it leads to
fluctuations in memory efﬁciency, as demonstrated in Fig. 4. This analysis considers one-at-a-time variations in each control ﬁeld parameter while
keeping the other parameters fixed at their optimal values. The resulting sensitivity in memory efﬁciency is expressed as a percentage, representing
the standard deviation in memory efﬁciency caused by each parameter's drift.

For example, in Fig. 6(a), we show the sensitivity of memory efﬁciency to variations in pulse area and delay. The largest sensitivity occurs in the
mixed ATS-EIT region, indicating that these parameters have the greatest impact on memory performance in this regi

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the focus of variance-based sensitivity analysis in quantum memory?
 response:   The focus of variance-based sensitivity analysis in
quantum memory is to determine the sensitivity of a quantum memory implementation to changes in intrinsic and extrinsic parameters while keeping
internal system parameters fixed.


Q1: What does Λ-type quantum memory refer to?
 response:   In the context of quantum physics, "Lambda-type" refers to a specific type of quantum
memory called /Lambda1-type quantum memory. This type of memory stores and retrieves photonic quantum states by manipulating the population transfer
between two hyperfine levels within an atom ensemble through a carefully controlled laser field. The name "Lambda-type" comes from the mathematical
representation used to describe the process, which involves the Lambda function, denoted by Λ(z).


Q2: Why are control field parameters crucial in Λ-type quantum memory systems?
 response:   Control field parameters are essential i

In [ ]:
json_file_name = 'VarianceQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'VarianceQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)